In [114]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import itertools
import glob
from datetime import date

In [115]:
## initiate file names for output results
current_results_file = f"nov23_results_joined_SL_{date.today().strftime('%Y%m%d')}"
all_results_file = f"all_results_joined_SL_{date.today().strftime('%Y%m%d')}"

# initate folder paths for data to read in
folder_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12"
prev_results = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/all_results_joined_SL_20231214.csv"
sl_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Master_Screening_Levels.xlsx"
pcb_arc_lookup_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/PCB_aroclor_lookup.csv"

PROCESS RESULTS SPREADSHEETS

In [116]:
file_extension = '*.xlsx'

# List all files with the specified extension in the folder
files = glob.glob(os.path.join(folder_path, file_extension))

results_df = []
length = 0
# Iterate through each file and read its content
for file_path in files:
    print(file_path)
    df = pd.read_excel(file_path)
    df.columns = df.columns.str.replace("_"," ")
    results_df.append(df)

results_df = pd.concat(results_df)
print(len(results_df))

/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311115 SGS EDD.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311175 SGS EDD Soil.xlsx


/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311175 SGS EDD water.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/B8755_EDD.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell Duwamish RCN 311018 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell Duwamish RCN 311115 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell UW (IAS) 311175 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/UWB UW IAS 312067 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/UWB UW IAS 312117 EIM.xlsx
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Scree

In [117]:
# remove any rows that were not field data
results_df =  results_df[results_df['Field Collection Start Date'].isna() == False]
len(results_df)

1720

In [118]:
print(results_df['Sample Matrix'].unique())
print(results_df['Sample Source'].unique())

# create new column of sample type based on sample matrix and sample source columns
results_df['Sample Matrix_clean'] = np.where(results_df['Sample Matrix']=='Aqueous', 'Water', results_df['Sample Matrix'])
results_df['Sample Matrix_clean'] = np.where(results_df['Sample Matrix'].str.contains('Solid'), 'Soil', results_df['Sample Matrix_clean'])
results_df['Sample Matrix_clean'] = np.where(results_df['Sample Source']=='Groundwater', 'Water', results_df['Sample Matrix_clean'])

['Aqueous' 'Solid' 'Solid/Sediment' 'Water']
[nan 'Soil' 'Groundwater']


In [119]:
# TODO: come back and figure out the PCBs with multiple numbers and make sure they are the right chemical name
# remove leading PCB-### from result parameter name to match screening level spreadsheet

def clean_fun(val):
    if 'PCB' in val:
        val = val.replace("/"," ").split(" ")[0]
    return val

results_df['Result Parameter Name_clean'] = results_df['Result Parameter Name'].apply(lambda x: clean_fun(x))

In [120]:
# remove trailing spaces in sample id
results_df['Sample ID'] = results_df['Sample ID'].str.strip()

# change sample ID DPS1 -> DPS-1
results_df['Sample ID'] = np.where(results_df['Sample ID'] == 'DPS1', 'DPS-1', results_df['Sample ID'])

# change sample ID SPB-0159-S-1 -> SPB-O159-S-1
results_df['Sample ID'] = np.where(results_df['Sample ID'] == 'SPB-0159-S-1', 'SPB-O159-S-1', results_df['Sample ID'])

# replace typo from 11/17 -> 11/07
results_df['Field Collection Start Date'] = np.where(results_df['Field Collection Start Date']=='2023-11-17 00:00:00', np.datetime64('2023-11-07'), results_df['Field Collection Start Date'])

In [121]:
# remove unnecessary columns from raw data
results_df = results_df[['Sample ID','Field Collection Start Date','Sample Matrix_clean','Sample Matrix','Sample Source',
                         'Result Parameter Name','Result Parameter Name_clean','Result Value', 'Result Value Units', 'Result Reporting Limit', 
                         'Result Reporting Limit Type', 'Result Detection Limit','Result Detection Limit Type', 'Result Data Qualifier']]

JOIN TABLES OF RESULTS TO MASTER SCREENING LEVELS FROM F&B

In [122]:
# create data frame of the screening levels for soils and
sl_soil_df = pd.read_excel(sl_path, sheet_name='Soil')
sl_water_df = pd.read_excel(sl_path, sheet_name='Water')

# concatenate to all screening levels
sl = pd.concat([sl_soil_df, sl_water_df])

In [123]:
# strip dioxin furans screening levels of their commas to match the results spreadsheet
sl['Chemical'] = np.where(sl['Chemical Group']== 'Dioxin Furans', sl['Chemical'].str.replace(',',''), sl['Chemical'])

# strip pcbs of their commas to match the results spreadsheet
sl['Chemical'] = np.where(sl['Chemical Group']== 'PCB', sl['Chemical'].str.replace(',',''), sl['Chemical'])

JOIN SCREENING LEVELS TO RESULTS

In [124]:
# create data frames of the raw data and the lookup
pcb_arc_lookup = pd.read_csv(pcb_arc_lookup_path)

In [125]:
pcb_arc_lookup['PCB Isomer'] = pcb_arc_lookup['PCB Isomer'].str.replace(",","")

In [126]:
# replace pcb names with the aroclor names to match with F&B results
sl_arc_join = pd.merge(sl, pcb_arc_lookup, how = 'outer', left_on = 'Chemical', right_on = 'PCB Isomer')
sl_arc_join['Chemical'] = np.where(sl_arc_join['Aroclor Name'].str.contains('aroclor', na=False),sl_arc_join['Aroclor Name'], sl_arc_join['Chemical'])

In [127]:
# join screening levels to the results
sl_results_join = pd.merge(sl_arc_join,results_df,how = 'outer', left_on = ['Chemical'], right_on = ['Result Parameter Name'])

In [128]:
# remove screening levels that do not have values
sl_results_join = sl_results_join[(sl_results_join['Screening Level Measurement']!='na')]
sl_results_join = sl_results_join[(sl_results_join['Screening Level Measurement']!='TBD')]
sl_results_join = sl_results_join[(sl_results_join['Screening Level Measurement']!='PQL')]

In [129]:
# calculate whether the screening levels have been exceeded
sl_results_join['SL_exceeded'] = np.where(sl_results_join['Screening Level Measurement'] < sl_results_join['Result Value'],'Y','N')

In [130]:
'''columns = ['Medium', 'Sample ID','Field Collection Start Date', 'Chemical Group', 'Chemical', 'Result Parameter Name', 'Scenario', 'Screening Level Type',
       'Screening Level Measurement', 'Result Value', 'Result Data Qualifier', 'SL Unit', 'Source', 'SL_exceeded']
'''

sl_results_join['SL_diff'] = sl_results_join['Result Value'] - sl_results_join['Screening Level Measurement']

In [131]:
# where the screening level is blank, replace exceedance with "no screening level identified"
sl_results_join['Screening Level Measurement'].fillna('No Screening Level Identified', inplace = True)
sl_results_join['SL_exceeded'] = np.where(sl_results_join['Screening Level Measurement']=='No Screening Level Identified','No Screening Level Identified', sl_results_join['SL_exceeded'])

In [132]:
sl_results_join.dropna(subset=['Sample ID'], inplace=True)
sl_results_join.rename(columns = {'Field Collection Start Date': 'DATE'}, inplace = True)
sl_results_join.to_csv(f'/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{current_results_file}.csv', index = False)

JOIN TO PREVIOUS SCREENING RESULTS

In [133]:
# join to previous results
prev_results_df= pd.read_csv(prev_results)

In [134]:
all_results_df = pd.concat([prev_results_df, sl_results_join])
all_results_df.to_csv(f'/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{all_results_file}.csv', index = False)